<a href="https://colab.research.google.com/github/MohamedeEssam/DL_projects/blob/main/Object_Detection_Distance_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np

# Load Yolo
print("LOADING YOLO")
net = cv2.dnn.readNet("weights\yolov2.weights", "cfg\yolov2.cfg")
#save all the names in file o the list classes
classes = []
with open("lib\coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
#get layers of the network
layer_names = net.getLayerNames()
#Determine the output layer names from the YOLO model 
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
print("YOLO LOADED")

LOADING YOLO
YOLO LOADED


## Function to calculate distance

In [1]:
import math

# Function to calculate the distance of object from the camera lense
def dist_calculator(startX, startY, endX, endY, box_width, box_height, img_w, img_h):

    x_3, y_3 = startX, endY - (box_height / 7)  # top left of the triangle
    # assumption: camera is rasied above the ground so considering 90% of the height of the image height
    x_1, y_1 = img_w / 2, 0.9 * img_h  # bottom of the triangle
    x_2, y_2 = endX, endY - (box_height / 7)  # top right of the triangle

    # find the angle between bottom and right point
    angle_x1_x2 = math.degrees(math.atan2(x_1 - x_2, y_1 - y_2))
    # find the angle between bottom and left point
    angle_x1_x3 = math.degrees(math.atan2(x_1 - x_3, y_1 - y_3))

    angle_right = 90 + angle_x1_x2
    angle_left = 90 - angle_x1_x3

    # total angle of view for the bench from bottom center point of the image.
    total_angle = angle_right + angle_left

    # Object length assumed to be 2 metersin millimeters. This value can automated, based on the type of bench used.
    object_length = 2000

    distance = (object_length * (1 / total_angle) * 57) / 1000

    # print(total_angle)
    # print(distance)
    return distance



## Calculate distance for image 

In [ ]:
# Capture frame-by-frame
img = cv2.imread("images\\bus.jpg")
#     img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# USing blob function of opencv to preprocess image
blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
    swapRB=True, crop=False)
#Detecting objects
net.setInput(blob)
outs = net.forward(output_layers)

# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)


            x1 = int(center_x - w * 0.5)  # Start X coordinate
            y1 = int(center_y - h * 0.5)  # Start Y coordinate
            x2 = int(center_x + w * 0.5)  # End X coordinate
            y2 = int(center_y + h * 0.5)  # End y coordinate



            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

#We use NMS function in opencv to perform Non-maximum Suppression
#we give it score threshold and nms threshold as arguments.
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
colors = np.random.uniform(0, 255, size=(len(classes), 3))
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        # calc distance
        distance = dist_calculator(x1, y1, x2, y2, w, h, width, height)

        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, "{0} Distance: {1} mm ".format(label, round(distance, 2)), (x, y -5),cv2.FONT_HERSHEY_SIMPLEX,
        1/2, color, 2)

cv2.imshow("Image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Calculate distance in real time

In [ ]:
video_capture = cv2.VideoCapture('images\\inference_videos\\video_1.mp4')
# video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    re,img = video_capture.read()
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # USing blob function of opencv to preprocess image
    blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416),
     swapRB=True, crop=False)
    #Detecting objects
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)


                x1 = int(center_x - w * 0.5)  # Start X coordinate
                y1 = int(center_y - h * 0.5)  # Start Y coordinate
                x2 = int(center_x + w * 0.5)  # End X coordinate
                y2 = int(center_y + h * 0.5)  # End y coordinate



                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    #We use NMS function in opencv to perform Non-maximum Suppression
    #we give it score threshold and nms threshold as arguments.
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_SIMPLEX=0
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]

            # calc distance 
            distance = dist_calculator(x1, y1, x2, y2, w, h, width, height)

            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, "{0} Distance: {1} mm ".format(label, round(distance, 2)), (x, y + 30), font, 1/2, color, 2)

    cv2.imshow("Image",cv2.resize(img, (800,600)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()